In [1]:
import torch
from torch.utils.data import DataLoader
from extractor.image import ImageFeatureExtractor 
from dataset import ImageLayout
from einops import rearrange
from PIL import Image

/workspace/joonsm/anaconda3/envs/CLDM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
a = ImageFeatureExtractor(backbone_name='resnet18').eval()

In [3]:
c= ImageLayout(type='val')

In [4]:
from torch.utils.data import DataLoader

In [58]:
loader = DataLoader(c, batch_size=128, shuffle=True)

In [64]:
sample = next(iter(loader))

In [65]:
box1= sample['box_cond'][0:64]
box2= sample['box_cond'][64:]

In [66]:
from loss import giou

In [68]:
giou(box1,box2).min()

tensor(-0.9176)

In [36]:
x,y,w,h

(tensor(-0.0625), tensor(0.1467), tensor(-0.9350), tensor(-0.9000))

In [18]:
import numpy as np

np.random.seed(0)
torch.manual_seed(0)

In [19]:
bsd = sample['image'].clone()

In [20]:
import torch

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [25]:
for i in range(5):
    bsd = sample['image'].detach()
    a(bsd).var()

In [27]:
a(bsd).var()

tensor(863239.6875, grad_fn=<VarBackward0>)

In [44]:
a(sample['image']).var()

tensor(0.5169, grad_fn=<VarBackward0>)

In [ ]:
extrs = ImageFeatureExtractor(backbone_name='resnet50')

In [ ]:
from dataset import ImageLayout

In [ ]:
train = ImageLayout()

In [ ]:
loader = DataLoader(train, batch_size=2)

In [ ]:
sample = next(iter(loader))

In [141]:
extrs(sample['image'])

NameError: name 'extrs' is not defined

In [ ]:
torch.Size([2, 256, 23, 23])

In [ ]:
sample['image']

In [ ]:
from PIL import Image

In [ ]:
train.frame_list[0]

'Town01/-6.-8.8inpaintseg/Town01.-6.-8.8.462.0.png'

In [ ]:
import os

In [ ]:
sss = os.path.join('/nas2/lait/1000_Members/jjoonvely/carla_new',train.frame_list[0])

In [ ]:
b= Image.open(sss)

In [ ]:
from torchvision import transforms

In [ ]:
transform = transforms.Compose([
        transforms.Resize((400,300)),
        transforms.ToTensor()
        ])

In [ ]:
import torch

In [ ]:
from einops import rearrange

In [ ]:
enc_output= torch.rand(2,2,2)

In [ ]:
enc_output= torch.rand(256,4,256)

In [ ]:

enc_output

tensor([[[0.1451, 0.0833]]])

In [ ]:
enc_output = rearrange(enc_output,'b c d -> b  (c d)' )

In [ ]:
enc_output.size()

torch.Size([256, 512])

In [ ]:


enc_output.squeeze(1).size()

torch.Size([1, 4])

In [75]:
import torch

# 파일에서 텐서를 불러오기
loaded_tensor = torch.load('aa.pt')

print(loaded_tensor)
print(f"Loaded tensor shape: {loaded_tensor.shape}")
print(f"Contains NaN: {torch.isnan(loaded_tensor).any()}")


tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         ...,

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, n

In [74]:
loaded_tensor.var()

tensor(0.5157, device='cuda:0')

In [73]:
src = transform(c)

TypeError: Unexpected type <class 'dataset.ImageLayout'>

In [36]:
ccc =a(src[:3,:,:].unsqueeze(0))

In [37]:
ccc.var()

tensor(0.5083, grad_fn=<VarBackward0>)

In [71]:
import torch

# 파일에서 텐서를 불러오기
loaded_tensor_ = torch.load('11.pt')

print(loaded_tensor_)
print(f"Loaded tensor shape: {loaded_tensor_.shape}")
print(f"Contains NaN: {torch.isnan(loaded_tensor_).any()}")

tensor([[[[-2.9630e+19, -3.3050e+19, -3.9364e+19,  ..., -3.4439e+19,
           -2.8057e+19, -2.4601e+19],
          [-3.2121e+19, -3.4341e+19, -3.8439e+19,  ..., -2.3144e+19,
           -1.8305e+19, -1.5684e+19],
          [-3.6720e+19, -3.6724e+19, -3.6730e+19,  ..., -2.2924e+18,
           -3.0150e+17,  7.7689e+17],
          ...,
          [-4.9540e+19, -5.6326e+19, -6.8854e+19,  ..., -8.2304e+19,
           -6.5890e+19, -5.7000e+19],
          [-6.0804e+19, -7.2353e+19, -9.3676e+19,  ..., -7.0358e+19,
           -5.5950e+19, -4.8145e+19],
          [-6.6905e+19, -8.1035e+19, -1.0712e+20,  ..., -6.3887e+19,
           -5.0565e+19, -4.3349e+19]],

         [[ 5.0388e+19,  5.0038e+19,  4.9390e+19,  ...,  2.5156e+19,
            1.2484e+19,  5.6193e+18],
          [ 5.7956e+19,  5.8625e+19,  5.9859e+19,  ...,  3.6168e+19,
            2.1697e+19,  1.3858e+19],
          [ 7.1927e+19,  7.4478e+19,  7.9187e+19,  ...,  5.6498e+19,
            3.8706e+19,  2.9068e+19],
          ...,
     

In [3]:
loaded_tensor_.var()

tensor(0.5164, device='cuda:0')

In [64]:
from LayoutDM import CLDM

In [65]:
a = CLDM()

In [68]:
a.extractor.body

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [69]:
def check_requires_grad(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"Parameter '{name}' requires gradient.")
            return False
    print("All parameters require_grad = False.")
    return True

In [70]:
check_requires_grad(a.extractor.body)

All parameters require_grad = False.


True